In [2]:
import torch
from torch import nn
import torchvision
import matplotlib.pyplot as plt

In [3]:
DATA_ROOT_DIR="FashionMnist"

In [4]:
train_data=torchvision.datasets.FashionMNIST(
root=DATA_ROOT_DIR,
train=True,
download=True,
transform=torchvision.transforms.ToTensor(),)
test_data=torchvision.datasets.FashionMNIST(
root=DATA_ROOT_DIR,
train=False,
download=True,
transform=torchvision.transform.ToTensor(),)

100.0%


Extracting FashionMnist\FashionMNIST\raw\train-images-idx3-ubyte.gz to FashionMnist\FashionMNIST\raw



100.6%
0.0%

Extracting FashionMnist\FashionMNIST\raw\train-labels-idx1-ubyte.gz to FashionMnist\FashionMNIST\raw



74.8%